# 1 - Introduction to the SampleData data platform

**This first User Guide page/notebook will**:

1. explain the motivation behind the design and implementation of this data platform
2. present the main features of the data format associated with the platform
3. list the prerequisite knowledge that we recommend to master before starting to use SampleData     
         
<div class="alert alert-info">

**Note:** This page/notebook does not include Python code to learn how to manipulate the SampleData class. However, it provides a complete overview of the scientific challenges that motivated the implementation of the class, which surely helps gaining insight in using the code and understand relevant applications.

</div>


## I - Why a data platform ? The data challenge 

### Context: the rise of 4D multi-modal material science

By exploiting the recent development of high-throughput and automated in-situ 3D or 2D imaging techniques, as well as highly parallel mechanical simulation codes on high performance computing (HPC) facilities, material scientists are already able to produce an unmatched quantity of rich datasets describing the physics of complex microstructures and architectures. These datasets will be gathered in large and shared databases that will be central to the collaborative study of deformation, damage and fracture of structural materials.

From the technical point of view, data management appears to be a major obstacle towards the realization of this new paradigm. Indeed, these rich databases gather data coming from a large number of materials, instruments and modeling/processing softwares. Material scientists aim at exploiting them through complex processing pipelines involving many steps and tools, including for instance visualization, statistical analysis, machine learning, model order reduction tools. Therefore, material scientists will be confronted with a profusion of data formats/shapes and software input/ouput requirements to handle. The following diagram illustrates the complexity of the resulting workflows that may be applied to each material sample studied following this paradigm. Each red arrows symbolize a data transfer operation, where complex format translation, storage and data alignement operations must be carried out. 

<img src="./Images/Introduction/Diagram_data_flow.png" width="75%">

If each scientist must deal with the increased technical complexity resulting from this situation, the efficiency of this new research paradigm will be considerably hindered. **Therefore, as for other science communities before, the construction of unified data management standards et tools is becoming a necessity for the materials science community.**  Their implementation should form a comprehensive platform, providing a simple and robust tool to support all the complexity involved with the management of those multimodal datasets for material scientists. Such a tool could become an interface facilitating interactions between the data and the scientists, and between the data and the various processing tools involved in research workflows. This perspective is schematized on the diagram below:

<img src="./Images/Introduction/Platform_diagram.png" width="50%">

The recent developments of the _pymicro_ Python package, in particular the creation of the `SampleData` class, are a contribution towards the construction of such a data platform. These efforts have been focused on addressing the following data related challenges:

### First challenge: Data multimodality

The first complexity in the construction of these datasets lies in the gathering under a unified format of data coming from various modalities. Here follows a quick overview of the most significant sources of data commonly used to analyze material microstructures. Handling them is a major goal of the `SampleData` class, with a particular focus on polycrystalline microstructures inherited from the _pymicro_ package main field of application. Thus, the data of interest will come from:

1. **Experimental techniques for material imaging** (2D/3D/4D data, can be applied _in-situ_)
    1. X-ray tomography 
    2. X-ray diffraction (near field and far field techniques)
    3. SEM scans
    4. EBSD scans
    5. Thermal imaging cameras
    
    
2. **Numerical simulations** (2D/3D/4D data)
    1. The Finite Element Method
    2. FFT based solvers
    3. Meshing tools
    
    
3. **Microstructure statistics**
    1. Microstructural features (grains/phases/particles morphological parameters, volume fraction....)
    2. Mechanical features (mean stress/strain state of particular microstructural features...)

### Second challenge: Data size

Besides their multimodal nature, the data of interest consist most of the time of very large datasets. Here follow a few orders of magnitude to illustrate the memory size of the data that must be dealt with.


1. **Tomographic datasets** 
    1. **X-ray tomography** : <br>
       *images of ~500 Mvoxels are a standard output. They are monochannels arrays of floats, that represent a few Go of raw data in memory*.
       
    2. **X-ray diffraction**: <br>
       *imaging around 1 mm*<sup>3</sup> *polycrystalline samples with a resolution close to 1 micron is within the reach of the main operating synchrotrons. DCT raw images for such scans represent a few To of data. Reconstructed microstructure are 3D images of around a billion of voxels, and have a typical size of a few Go to a few tens of Go.*
    
    
2. **Numerical simulations** (2D/3D/4D data)
    1. **Finite Element Method** (FEM) <br>
       Meshing large microstructure obtained with high-resolution 3D imaging techniques (see 1.) usually results in FEM meshes of at least tens of millions to hundreds of millions elements. Storing mechanical fields over similar meshes require to store tens of millions of floating point number for each scalar field. In the end, simulation outputs will represent tens of Go for each snapshot at which simulation fields are outputed.
       
    2. **FFT based solvers** <br>
       FFT based simulations operate directly on microstructure images, typically of the size of images discussed in 1. just above. Just like FEM simulations, they will result in storing for each scalar field hundreds of millions to billions of floating point values, for each snapshot. Hence, simulation outputs for one microstructure will represent tens to hundreds of Go of data. 
       
Because of these large data sizes, some data management strategies become mandatory:
* efficient data compression, to save memory space and enable the storage of large databases
* automation of data processing (manually processing  datasets of this size becomes practically impossible)

### Third challenge: Open Data

As Big Data related methods increase the complexity of the research process, open data access and complete transparency of data processing steps are mandatory to ensure the reproducibility of scientific results. In addition, the scientific value of the produced data increases when it can be accessed by scientist all around the world all free to enrich and exploit them with their specific expertise. In fact, the value of the scientific data often extends beyond the scope of the first application that initially motivated its production. Finally, constructing open databases also allows to gather data produced of the same nature but produced by different teams.

Hence, a key challenge for the data management tool is to allow to add and retrieve all sort of metadata for each element of those complex datasets, in order to give all the relevant information on the nature of the data and the processing steps apply to it (material nature and specifications, material preparation process, experimental procedure, numerical treatments applied, simulation algorithms and material models used....). 


## II - The SampleData platform

The *pymicro* package is used on a daily basis by material scientists teams involved in the development of the data paradigm presented in section I. Its original design, that relied on classical Python and Numpy data objects to handle material data, dit not allow to efficiently handle the multi-modal 4D material science data challenges. Therefore, an new  subpackage has been introduced within *pymicro*, the `pymicro.core` subpackage, to serve as *pymicro*'s data platform. 

This `pymicro.core` package has been designedtto be independent of the rest of the *pymicro* code. Its aim is to serve as a prototype implementation of what could constitute a large-scale data platform the material microstructure science community. In the future, it is likely that it will become a fully independent package, and hence an external *pymicro* dependency. In that case, it will most likely be name *SampleData*, as its main class object.

`SampleData` is a Python class designed to implement two essential functions for the management of multimodal material data sets within the Pymicro code:
1. A unified file format and data model able to handle all data modalities presented below, within the same dataset.
2. A high-level interface allowing interactions with datasets for user and external numerical tools while hiding all the data management technical complexity

As its name suggests, it is designed to associate a dataset file for each material sample, whose data must be collected, stored and/or processed. The class then behave as a high-level interface allowing to interact with the data. Its main functionalities are:
* Add, remove, and organize easily data of various types, formats, shapes into the same dataset
* Add, remove and organize easily light metadata for all elements of a dataset
* get simple or detailed information on the dataset content
* allow easy visualization of spatially organized data (images, maps, measured or simulated fields)
* allow a flexible and efficient compression of each element of a dataset
* a framework to automate interface between the datasets and data processing or numerical simulation tools
* a framework to derive classes from SampleData that are specialized to a specific type of material samples, such as the `Microstructure` class of the `pymicro.crystal.microstructure` module, dedicated to polycrystalline samples
* ensure convergence of spatially organized data 

Regarding this last point, the "*convergence*" of data refers to the conversion into a unified data model of all data representing microstructure geometries, of mechanical fields (measured or simulated), of different origins. This allows to directly visualize and process them together with the same tools. 

The `SampleData` class is thus the interface that the data platform user has to manipulate in order to access those functionalities, and interact with the dataset. Its detailed presentation is the subject of the series of notebooks of which this document is only the first. The rest of the section is dedicated to the presentation of the file format and data model that is implement by the `SampleData` class.

###  The SampleData file format

Even if its implementation does not require it, the `SampleData` class has been designed to be used with the following convention:  **1 dataset per material sample**. Each dataset is associated with a pair of files:

1. a HDF5 file 
2. a XDMF file

HDF5 is a library and file format that supports an unlimited variety of datatypes, and is designed for flexible and efficient I/O and for high volume and complex data.  It is a hierarchical file format in the sense that it allows to gather multiple data within a file and create links to organize them in relation to each other, creating an
internal file hierarchy, replicating a file system with directories and files within one file. HDF5 stores data with a binary encoding, and is compatible with many of the most powerfull compression algorithms available. It also allows to add text metadata to all data elements in the file, called *HDF5 attributes*. **The HDF5 file, within the SampleData file pair, is the actual data set: it contains all data and metadata related to the material sample.**

XMDF is an extensible Data Model to exchange scientific data between High Performance Computing codes and numerical tools. It has been practically designed for spatially organized data, *i.e.* fields, used as inputs and outputs by simulation tools. Practically, it is an XML file containing the description, for each grid supporting the fields, of its topology, its geometry, its dimensions, as well as the data arrays, types and dimensions of field defined on the grid. Rather than writing the heavy data describing gris and fields in to the XML file, the XDMF syntax allow to refer to arrays stored in HDF5 data files. In that case, and **in the case of the `SampleData` file pair, the XDMF file can be seen as an external metadata file allowing to interpret raw binary data stored in a HDF5 file as spatially organized data**. 

One important feature of thos HDF5/XDMF file pairs, is that they are a rather standard format. In particular, a reader for this format is implemented in the powerfull and popular 3D visualization software Paraview. **The `SampleData` class ensures that both HDF5 and XML are always synchronized together and with the content of the dataset.** This means that whenever users add spatially organized data into a `SampleData`  HDF5 dataset, they can simply visualize their dataset content by opening with Paraview the XMDF file.

###  The SampleData Data Model

In addition to the HDF5/XDMF file pair, `SampleData` datasets comply with a specific data model. This section provides an quick overview of the different elements of this data model. Each of these elements will be extensively detailed in the next Notebooks of this User Guide. 

This data model is built on top of the HDF5 data model, whose primary objects are:

* **Groups**: a data structure that can be linked to other groups or datasets, and is used to organized data objects. They can been seen as the 'directories' of a HDF5 dataset. Every HDF5 file contains a root group that can contain other groups.
* **Data arrays** or **Nodes**: arrays of data that can have different types/shapes, and are attached to a Group.
* **Attributes**: Name/Value pairs that can take any form as long as they remain small. It is the official way to store metadata into HDF5 files. Both Groups and Datasets can hold as many Attributes as required.

The `SampleData` data model introduces two types of particular HDF5 Groups. They are dedicated to the representation of spatially organized data. Such data consist in geometrical grid supporting fields. These grids can be regular gris, or have more complex topologies. Those two situations correspond to the 2 Group types in the data model, that are:

* **Image Groups** are HDF5 groups designed to store data describing 2D or 3D images, *i.e.* regular grids supporting scalar or tensorial fields. They are used for instance, to store data coming from SEM/EBSD or X-ray tomography imaging, or FFT simulation results. They are actually 3 types of image groups:
    - `2DImage` groups: represent two dimensional images, *i.e.* grids of $(N_x,N_y)$ pixels
    - `3DImage` groups: represent three dimensional images, *i.e.* grids of $(N_x,N_y, N_z)$ voxels
    - `emptyImage` groups: represent image groups that do not yet support any data and topology yet
* **Mesh Groups** are HDF5 groups designed to store data describing 2D or 3D meshes, *i.e.* grids described by node coordinates and elements connectivities, supporting scalar or tensorial fields. They can be used to store data coming from finite element simulations, CAD designs etc...They are actually 3 types of image groups:
    - `2DMesh` groups: represent two dimensional meshes, *i.e.* grid of nodes defined by their $(X_i,Y_i)$ coordinate pairs
    - `3DMesh` groups: represent three dimensional meshes, *i.e.* grid of nodes defined by their $(X_i,Y_i,Z_i)$ coordinate pairs
    - `emptyMesh` groups: represent mesh groups that do not yet support any data and topology yet

In addition, the data model introduces two additional types of HDF5 **Nodes**:

* **Structured Tables** are heterogeneous and bidimensional data arrays, *i.e.* that may contain data of different types (integers, floats, strings, sub arrays...) within the same row, all rows having the same format. Each column of those arrays can have a specific name. Those arrays are the in-memory equivalent of [Numpy structured arrays](https://numpy.org/doc/stable/user/basics.rec.html)
* **Fields** are specific data arrays that must belong to a grid group (Image or Mesh Group), and whose shape and dimensions must comply with the grid topology. They are used to store and manipulate spatially organized arrays, that represent mechanical fields (for instance, displacement or temperature fields, microstructure phase maps, EBSD orientation maps....)

Those specific data objects introduce by `SampleData` all have a specific data model, associated metadata, and a specific interface. They are all reviewed in details by a specific tutorial Notebook of this user guide. 

### Data Model example

To illustrate what `SampleData` datasets may look like, a virtual example of a polycrystalline material sample dataset is represented in the schematic diagram below:

<img src="./Images/Introduction/DataModel-figure1.png" width="80%">

Groups are used to organize data into coherent categories. Metadata are represented in green, and datasets in red. As it appears on the diagram, Groups can be dedicated only to organize metadata, for instance to document material nature, composition, elaboration process, or the experimental set-up used for imaging and mechanical tests. The *Macro Data*  and *Statistics* Groups will typically contain simple *arrays* datasets or *structured tables*, to store mechanical tests macroscopic outputs (loading curves) or some statistics on the microstructure geometry or mechanical state. The *Heavy Data* group will typically contain the spatially organized data, coming from 3D or 2D in-situ imaging techniques, and from numerical simulation softwares. It will most likely contain *Image or Mesh Groups*, filled with *Fields* datasets.

## III - SampleData dependencies and pre-requisites

As `SampleData` is a rather complex class, relying on several libraries and packages, it is strongly adviced to take a look at and learn the basics of the following tools/packages:

* Obviously, basic notions of Python (Python syntax, data types, namespaces, imports, classes, working with class methods, Python interpreters)
* [the HDF5 data model](https://portal.hdfgroup.org/display/HDF5/Introduction+to+HDF5)
* [the XDMF data model](https://www.xdmf.org/index.php/XDMF_Model_and_Format)
* [the Numpy package](https://numpy.org/doc/stable/index.html)
* [the Paraview visualization software](https://docs.paraview.org/en/latest/)

`SampleData` HDF5 interface is based on the Python package [Pytables](https://www.pytables.org/). It handles the management of the HDF5 file and the compression of datasets. Knowing its basics can be a valuable help when using SampleData.
         
<div class="alert alert-warning">

**Warning:** In the series of notebooks that make up this `SampleData` user guide, a minimal knowledge of these elements will be assumed.

</div>


Now that the relevant context and description of the data platform has been set, you can start to learn how to use it with the second Notebook of this user guide !